In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%gui asyncio
%config IPCompleter.greedy=True
import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
from apache_beam.runners.direct import direct_runner
from apache_beam.testing.test_stream import TestStream
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import DebugOptions
from apache_beam.portability.api import beam_fn_api_pb2
from apache_beam.portability.api import beam_fn_api_pb2_grpc
import time
import cProfile
import grpc

/usr/local/google/home/srohde/Workdir/beam/sdks/python/apache_beam/__init__.py:84: UserWarning: Some syntactic constructs of Python 3 are not yet fully supported by Apache Beam.
  'Some syntactic constructs of Python 3 are not yet fully supported by '


In [2]:
from IPython.display import display
play_button = widgets.Button(description="Play")
pause_button = widgets.Button(description="Pause")
step_button = widgets.Button(description="Step")

output = widgets.Output()
display(play_button, output)
display(pause_button, output)
display(step_button, output)

def on_play_button(b):
    with output:
        channel = grpc.insecure_channel('localhost:12345')
        stub = beam_fn_api_pb2_grpc.InteractiveStreamStub(channel)
        stub.Start(beam_fn_api_pb2.StartRequest())
        
def on_pause_button(b):
    with output:
        channel = grpc.insecure_channel('loIs an explicit API that the user will know exactly what to expect when they rerun a pipelinecalhost:12345')
        stub = beam_fn_api_pb2_grpc.InteractiveStreamStub(channel)
        stub.Pause(beam_fn_api_pb2.PauseRequest())
        
def on_step_button(b):
    with output:
        channel = grpc.insecure_channel('localhost:12345')
        stub = beam_fn_api_pb2_grpc.InteractiveStreamStub(channel)
        stub.Step(beam_fn_api_pb2.StepRequest())

play_button.on_click(on_play_button)
pause_button.on_click(on_pause_button)
step_button.on_click(on_step_button)



Button(description='Play', style=ButtonStyle())

Output()

Button(description='Pause', style=ButtonStyle())

Output()

Button(description='Step', style=ButtonStyle())

Output()

cubes produced 0
square produced 0
got request token: 1



In [3]:
from functools import partial
import threading
pipeline_options = PipelineOptions()
pipeline_options.view_as(StandardOptions).streaming = True

def printer(name, x):
    print(name, "produced", x)
    return x

def named_printer(name):
    return partial(printer, name)
    

def run():
    runner = interactive_runner.InteractiveRunner(render_option='mute', endpoint='localhost:12345')  # render_option could also be 'mute', 'text'
    p = beam.Pipeline(runner=runner, options=pipeline_options)
    pc = p | 'read' >> (TestStream(endpoint='localhost:12345'))
        
    squares = pc | 'Square' >> beam.Map(lambda x: x*x) | beam.Map(named_printer('square'))
    cubes = pc | 'Cube' >> beam.Map(lambda x: x**3) | beam.Map(named_printer('cubes'))
    result = p.run()
    
t = threading.Thread(target=run)
t.daemon = True
t.start()

Running...

Using 0 cached PCollections
Executing 5 of 5 transforms.

Map(functools.partial(<function printer at 0x7f0bd815e598>, 'cubes')) produced {}

read produced {}

Map(functools.partial(<function printer at 0x7f0bd815e598>, 'square')) produced {}

Square produced {}

Cube produced {}

got request 
